# Imports

In [56]:
# For data manipulation and analysis
import pandas as pd  

# For creating graphical user interfaces
import tkinter as tk  

# For displaying message boxes
from tkinter import messagebox  

# For interacting with the operating system
import os  

# For working with dates and times
from datetime import datetime  

# For calendar-related functions and operations
import calendar  


## Select the month or week you'd like to capture in the report
A window will pop up to select the desired report after running the full script.

In [57]:

# (week number, year) 
WeekNumber = (24,2024)

# (month number, year) 
MonthYear = (6,2024)

# Gathering Files

All suppression files downloaded from Vendor Central should be stored in the same folder.

Example of a suppression file name: amazon_df_suppression_bulk_2024-03-26_12_23_13_UTC_dataset_0

The file name can be used to extract the date, which can then be stored in a column. This will allow aggregation by time frame.

In [58]:
# creating folder path
folder_path = "C:/Users/CraigJustin/Documents/Amazon Suppressions"

# Create a list of all file names in the folder, excluding 'desktop.ini'
files = [f for f in os.listdir(folder_path) if f != 'desktop.ini']

# Create a list of absolute file paths for each file
supp_path_list = [os.path.join(folder_path, file) for file in files]

# Create a list of DataFrames for each file in the folder, assigning the file name as a new column 'Date'
df_list = [pd.read_csv(file).assign(Date=file) for file in supp_path_list]

# Display the number of files captured in the folder
print(len(df_list))


86


In [59]:
# Create a consolidated DataFrame by concatenating all DataFrames in df_list along the rows
full_df = pd.concat(df_list, axis=0)

# Display the number of rows and columns in the consolidated DataFrame
full_df.shape


(2602830, 8)

# Cleaning Sheet

In [60]:
# cleaning sku column
full_df['SKU'] = full_df['SKU'].str.replace('="', '').str.replace('"', '')

# creating a subset dataframe
full_df = full_df[['SKU','ASIN','Suppression reason','Date']]

# extracting date from file name
full_df['Date'] = full_df['Date'].str.replace(folder_path, '')
full_df['Date'] = full_df['Date'].str.replace('/', '')
full_df['Date'] = full_df['Date'].str.replace("\\", "")
full_df['Date'] = full_df['Date'].str.replace('amazon_df_suppression_bulk_', '')
full_df['Date'] = full_df['Date'].str.split('_').str[0]

# reformatting date column to mm-dd-yyyy and sorting by date asc
full_df['Date'] = pd.to_datetime(full_df['Date'])
full_df['Date'] = full_df['Date'].dt.strftime('%m-%d-%Y')
full_df = full_df.sort_values(['Date'])

# resetting the index
clean_df = full_df.reset_index(drop=True)


In [61]:
# creating a month, month name, year and week number column
clean_df['Month Number'] = pd.to_datetime(clean_df['Date']).dt.month
clean_df['Year'] = pd.to_datetime(clean_df['Date']).dt.year
clean_df['Week Number'] = pd.to_datetime(clean_df['Date']).dt.isocalendar().week

In [62]:
# creating a modified data frame to drop duplicates as folder may have multiple files from the same day
modified_df = clean_df.drop_duplicates()

In [63]:
# filtering to given week and year, and trimming to show relavent columns, and analyzing count for each day
modified_df_week = modified_df[(modified_df['Week Number']==WeekNumber[0]) & (modified_df['Year']==MonthYear[1])]
modified_df_week = modified_df_week[['SKU','ASIN','Date']].drop_duplicates()

# filtering to given month and year, and trimming to show relavent columns, and analyzing count for each day
modified_df_monthyear = modified_df[(modified_df['Month Number']==MonthYear[0]) & (modified_df['Year']==MonthYear[1])]
modified_df_monthyear = modified_df_monthyear[['SKU','ASIN','Date']].drop_duplicates()

In [64]:
# creating a weekly_count data frame to count how many days an asin appears suppressed for the week, then creating a suppression rate column based on the count.
weekly_count = modified_df_week.groupby('ASIN').count().reset_index().sort_values('SKU')
weekly_count['Suppression Rate'] = weekly_count['SKU']/weekly_count['SKU'].max()

# creating a monthly_count data frame to count how many days an asin appears suppressed for the month, then creating a suppression rate column based on the count.
monthly_count = modified_df_monthyear.groupby('ASIN').count().reset_index().sort_values('SKU')
monthly_count['Suppression Rate'] = monthly_count['SKU']/monthly_count['SKU'].max()

## Read Vendor Catalog

The sheet that I'm reading in represents all active SKUs on Amazon. This helps to narrow down the report to relevant SKUs in the current catalog.

In [65]:
# pulling in monk report F521 for current active items (must download first)
filepath = r'vendor_catalog.txt'
vendor_df = pd.read_csv(filepath)

vendor_df.columns = ['Key', 'Vendor SKU', 'Customer SKU', 'Description']
vendor_df = vendor_df[['Vendor SKU', 'Customer SKU']]

In [66]:
# merging fast codes data frame with weekly count data frame so each active sku has a suppression rate 
active_skus_suppress_rate_weekly = vendor_df.merge(weekly_count,how='left', left_on='Customer SKU', right_on='ASIN')
active_skus_suppress_rate_monthly = vendor_df.merge(monthly_count,how='left', left_on='Customer SKU', right_on='ASIN')

In [67]:
# removing irrelavant columns, filling n/a values with 0 (as n/a would represent that they weren't suppressed for that week)
active_skus_suppress_rate_weekly = active_skus_suppress_rate_weekly[['Vendor SKU','Customer SKU','Suppression Rate']]
active_skus_suppress_rate_weekly['Suppression Rate'] = active_skus_suppress_rate_weekly['Suppression Rate'].fillna(0)

# removing irrelavant columns, filling n/a values with 0 (as n/a would represent that they weren't suppressed for that month)
active_skus_suppress_rate_monthly = active_skus_suppress_rate_monthly[['Vendor SKU','Customer SKU','Suppression Rate']]
active_skus_suppress_rate_monthly['Suppression Rate'] = active_skus_suppress_rate_monthly['Suppression Rate'].fillna(0)

# Exporting Report

In [68]:
# To Select which report to download (monthly or weekly)
file_path = "C:/Users/CraigJustin/Documents/"

def export_weekly_report():
    try:
        now = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
        filename = file_path + "amazon_suppression_weekly_" + str(WeekNumber[0]) + "-" + str(WeekNumber[1]) +  "-" + now + ".csv"
        active_skus_suppress_rate_weekly.to_csv(filename, index=False)
        messagebox.showinfo("Success", f"Weekly report saved as {filename}")
    except Exception as e:
        messagebox.showerror("Error", f"Failed to save weekly report: {str(e)}")
    root.destroy()

def export_monthly_report():
    try:
        now = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
        filename = file_path + "amazon_suppression_monthly_" + calendar.month_name[MonthYear[0]] + "-" + str(MonthYear[1]) + "-" + now + ".csv"
        active_skus_suppress_rate_monthly.to_csv(filename, index=False)
        messagebox.showinfo("Success", f"Monthly report saved as {filename}")
    except Exception as e:
        messagebox.showerror("Error", f"Failed to save monthly report: {str(e)}")
    root.destroy()

# Tkinter GUI setup
root = tk.Tk()
root.title("Report Downloader")
root.geometry("400x200")  # Set the window size to 400x200 pixels

# Create and place buttons
weekly_button = tk.Button(root, text="Download Weekly Report", command=export_weekly_report)
weekly_button.pack(pady=10)

monthly_button = tk.Button(root, text="Download Monthly Report", command=export_monthly_report)
monthly_button.pack(pady=10)

# Run the Tkinter main loop
root.mainloop()
